In [1]:
# Let's execute the cell below to display information about the GPUs running on the server.
!nvidia-smi

# Finally, execute the following cell to show the version of TensorFlow that is used in this lab.
import tensorflow as tf; 
print(tf.__version__)

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import spacy # NLU library
spacy.load('en_core_web_sm')
from spacy.lang.en import English

import en_core_web_sm
nlp = en_core_web_sm.load()

import en_core_web_lg # Large SpaCy model for English language
import numpy as np
import re # regular expressions

from collections import defaultdict
from sklearn.svm import SVC # Support Vector Classification model

ModuleNotFoundError: No module named 'spacy'

In [17]:
output_format = "User: {input}\nSITA: {output}\n" + "_"*50

## 1) Use exact matches¶

hard-coded exact questions

In [14]:
# hard-coded exact questions
responses_exact = {
    "what would you like to eat tonight?": "Pasta with salmon and red pesto please!",
    "what time will you be home tonight?": "I will be home around 6 pm.",
    "default": "I love you too!"
}

def respond_exact(text):
    response = responses_exact.get(text.lower(), responses_exact['default'])
    return(output_format.format(input=text, output=response))

In [19]:
print(respond_exact("What would you like to eat tonight?"))
print("_"*50)
print(respond_exact("What time will you be home tonight?"))
print("_"*50)
print(respond_exact("I just found out my boss is leaving the company."))

User: What would you like to eat tonight?
SITA: Pasta with salmon and red pesto please!
__________________________________________________
__________________________________________________
User: What time will you be home tonight?
SITA: I will be home around 6 pm.
__________________________________________________
__________________________________________________
User: I just found out my boss is leaving the company.
SITA: I love you too!
__________________________________________________


## 2) Pattern Matching

In [21]:
# Define keywords that can help determine the intent 
intent_keywords = {
    'dinner_preference': ['eat', 'dinner', 'food', 'cook', 'craving'],
    'arrival_time': ['time', 'when', 'get here', 'be home']
}
# Create a dictionary of patterns
patterns = {intent: re.compile('|'.join(keys)) for intent, keys in intent_keywords.items()}

# Define a function to find the intent of a message
def get_intent_re(message):
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            return(intent)
    else:
        return('default')

responses_re = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "default":"I like you too!"
}

def respond_re(text):
    response = responses_re.get(get_intent_re(text))
    return(output_format.format(input=text, output=response))

In [22]:
print(respond_re("what would you like to eat tonight?"))
print(respond_re("what time will you be home tonight?"))
print(respond_re("I just food out my boss is leaving the company."))

User: what would you like to eat tonight?
SITA: Pasta with salmon and red pesto please!
__________________________________________________
User: what time will you be home tonight?
SITA: I will be home around 6 pm.
__________________________________________________
User: I just food out my boss is leaving the company.
SITA: Pasta with salmon and red pesto please!
__________________________________________________


## 3) Machine Learning

In [23]:
# Create training data
training_sentences = [
    "What would you like to have for dinner?",
    "What do you want to eat tonight?",
    "I don't know what to cook tonight.",
    "Do you have any cravings?",
    "Can I get you something to eat?", 
    "What time will you be home?",
    "How much longer will you be?",
    "When can we expect you to get here?",
    "What's taking you so long?",
    "At what hour will you be here?"
    
]
training_intents = [
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "arrival_time",
    "arrival_time",
    "arrival_time",
    "arrival_time",
    "arrival_time"   
]

In [27]:
# this may take a couple of seconds
nlp = en_core_web_lg.load()

In [28]:
# Initialize the array with zeros: X
X_train = np.zeros((len(training_sentences), 
              nlp('sentences').vocab.vectors_length))

for i, sentence in enumerate(training_sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X_train[i, :] = doc.vector

In [29]:
# Create a support vector classifier
clf = SVC(C=1, gamma="auto", probability=True)

# Fit the classifier using the training data
clf.fit(X_train, training_intents)

#Yes, a lot can be done here to check / improve model performance! We will leave that for another day!

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [30]:
def get_intent_ml(text):
    doc = nlp(text)
    return(clf.predict([doc.vector])[0])

In [31]:
# Figure out how to deal with it.
responses_ml = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "default":"I like you too!"
}

def respond_ml(text):
    response = responses_ml.get(get_intent_ml(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [32]:
print(respond_ml("what would you like to eat tonight?"))
print(respond_ml("what time will you be home tonight?"))
print(respond_ml("l"))

User: what would you like to eat tonight?
SITA: Pasta with salmon and red pesto please!
__________________________________________________
User: what time will you be home tonight?
SITA: I will be home around 6 pm.
__________________________________________________
User: l
SITA: Pasta with salmon and red pesto please!
__________________________________________________


In [33]:

def get_intent_ml_2(text):
    """
        Returns the intent from a given text, unless the model is not sure, in which case 'default' is returned
    """
    doc = nlp(text)
    max_proba = max(clf.predict_proba([doc.vector])[0])
    if(max_proba == 0.5):
        return('default')
    else:
        return(clf.predict([doc.vector])[0])

def respond_ml_2(text):
    response = responses_ml.get(get_intent_ml_2(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [34]:
print(respond_ml(  'flowers'))
print(respond_ml_2('flowers'))

User: flowers
SITA: Pasta with salmon and red pesto please!
__________________________________________________
User: flowers
SITA: I like you too!
__________________________________________________


In [35]:
def get_all_entities(text):
    """
        Get all entities in a given text, in a text: label_ dictionary
    """
    doc = nlp(text)
    
    d = defaultdict(list)
    for ent in doc.ents:
        d[ent.label_].append(ent.text)
    return(d)

In [36]:
test_ents = get_all_entities('what would you like to eat tonight?, or next tuesday or wednesday fish football Bengals')
print(sorted(test_ents.items()))

[('DATE', ['next tuesday', 'wednesday']), ('GPE', ['Bengals']), ('TIME', ['tonight'])]


In [37]:
policy = {
    ("dinner_preference", "time and date"): "I want to eat pasta",
    ("dinner_preference", "time only"): "I want to eat pasta",
    ("dinner_preference", "date only"): "I want to eat pasta",
    ("dinner_preference", "none"): "When?",
    ("arrival_time", "time and date"): "I will be home at six",
    ("arrival_time", "time only"): "I will be home at six",
    ("arrival_time", "date only"): "I will be home at six",
    ("arrival_time", "none"): "When?",
    ("default", "none"): "What do you want?",
}



In [38]:
def respond_ml_3(text):
    """Check for specification of date and time
        If not specified, ask for clarification
    """
    intent = get_intent_ml_2(text)
        
    if intent != 'default':
        entities = get_all_entities(text)
        if 'TIME' in entities and 'DATE' in entities:
            specification = 'time and date'
            time = ' and '.join(entities['DATE']) + ' at ' + ' and '.join(entities['TIME'])
        elif 'TIME' in entities:
            specification = 'time only'
            time = ' and '.join(entities['TIME'])
        elif 'DATE' in entities:
            specification = 'date only'
            time = ' and '.join(entities['DATE'])
        else:
            specification = 'none'
            time = ""
    else:
        specification = 'none'
        time = ""
    
    response = policy.get((intent, specification)) + ' ' + time
    return(output_format.format(input=text, output=response))

In [39]:
print(respond_ml_3('what would you like to eat next wednesday thursday and friday?'))

User: what would you like to eat next wednesday thursday and friday?
SITA: I want to eat pasta thursday and friday
__________________________________________________


In [42]:
import re
# Define replace_pronouns()
def replace_pronouns(message):
    message = message.lower()
    
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me', 'you', message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my', 'your' ,message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your','my',message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub('you', 'me',message)

    return message

print(replace_pronouns("my last birthday"))
print(replace_pronouns("when you went to Florida"))
print(replace_pronouns("I had my own castle"))

your last birthday
when me went to florida
i had your own castle
